# Introduction to PDDL

In the first example, we looked at manually defining a planning problem.

However, there exist entire modeling languages to more easily define these.
The most common one is Planning Domain Description Language, or PDDL.

Unified Planning offers a PDDL reader that you can use to directly read these files.

In [9]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *

reader = PDDLReader()
pddl_problem = reader.parse_problem(
    "pddl/pick_place_domain.pddl",
    "pddl/pick_place_problem_01.pddl",
)
print(pddl_problem)

problem name = pick_and_place_problem_01

types = [bin, part]

fluents = [
  bool robot_at[b=bin]
  bool part_at[p=part, b=bin]
  bool holding[p=part]
  bool gripper_empty
]

actions = [
  action move(bin b_from, bin b_to) {
    preconditions = [
      robot_at(b_from)
    ]
    effects = [
      robot_at(b_from) := false
      robot_at(b_to) := true
    ]
  }
  action pick(part p, bin b) {
    preconditions = [
      (robot_at(b) and part_at(p, b) and gripper_empty)
    ]
    effects = [
      part_at(p, b) := false
      holding(p) := true
      gripper_empty := false
    ]
  }
  action place(part p, bin b) {
    preconditions = [
      (robot_at(b) and holding(p))
    ]
    effects = [
      holding(p) := false
      gripper_empty := true
      part_at(p, b) := true
    ]
  }
]

objects = [
  bin: [bin_1, bin_2, bin_3]
  part: [part_1, part_2]
]

initial fluents default = [
  bool robot_at[b=bin] := false
  bool part_at[p=part, b=bin] := false
  bool holding[p=part] := false
  bool 

Once you have read in domain and problem files from PDDL, you can perform planning in the same way as manually defined domains!

In [10]:
with OneshotPlanner(name="tamer") as planner:
    result = planner.solve(pddl_problem)
    if result.plan is None:
        print("Found no plan :(")
    else:
        print(result.plan)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_2334124/2296574505.py`, you are using the following planning engine:
  * Engine name: Tamer
  * Developers:  FBK Tamer Development Team
  * Description: Tamer offers the capability to generate a plan for classical, numerical and temporal problems.
  *              For those kind of problems tamer also offers the possibility of validating a submitted plan.

SequentialPlan:
    move(bin_3, bin_2)
    pick(part_2, bin_2)
    move(bin_2, bin_1)
    place(part_2, bin_1)
    pick(part_1, bin_1)
    move(bin_1, bin_3)
    place(part_1, bin_3)
    move(bin_3, bin_2)


By defining your domains and problems in separate PDDL files, the amount of code written here is quite minimal!

The idea is that you can reuse the same domain file along with any arbitrary number of problem files.

Feel free to make your own additional problem files in the `pddl` subfolder, and see how things go!